## Informações do ambiente computacional

In [76]:
!more /proc/cpuinfo &> processador.txt
!more processador.txt | grep model

model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz


In [77]:
!more processador.txt | grep 'cpu core'

cpu cores	: 1
cpu cores	: 1


## Implementação sequencial

In [78]:
%%writefile difusao.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 2000  // Tamanho da grade
#define T 500 // Número de iterações no tempo
#define D 0.1  // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

void diff_eq(double **C, double **C_new) { //diff_eq(double C[N][N], double C_new[N][N]) {
    for (int t = 0; t < T; t++) {
        for (int i = 1; i < N - 1; i++) {
            for (int j = 1; j < N - 1; j++) {
                C_new[i][j] = C[i][j] + D * DELTA_T * (
                    (C[i+1][j] + C[i-1][j] + C[i][j+1] + C[i][j-1] - 4 * C[i][j]) / (DELTA_X * DELTA_X)
                );
            }
        }
        // Atualizar matriz para a próxima iteração
        double difmedio = 0.;
        for (int i = 1; i < N - 1; i++) {
            for (int j = 1; j < N - 1; j++) {
                difmedio += fabs(C_new[i][j] - C[i][j]);
                C[i][j] = C_new[i][j];
            }
        }
        if ((t%100) == 0)
          printf("interacao %d - diferenca=%g\n", t, difmedio/((N-2)*(N-2)));
    }
}

int main() {

    // Concentração inicial
    double **C = (double **)malloc(N * sizeof(double *));
    if (C == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (int i = 0; i < N; i++) {
      C[i] = (double *)malloc(N * sizeof(double));
      if (C[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }
    for (int i = 0; i < N; i++) {
      for (int j = 0; j < N; j++) {
        C[i][j] = 0.;
      }
    }

    // Concentração para a próxima iteração
    double **C_new = (double **)malloc(N * sizeof(double *));
    if (C_new == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (int i = 0; i < N; i++) {
      C_new[i] = (double *)malloc(N * sizeof(double));
      if (C_new[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }
    for (int i = 0; i < N; i++) {
      for (int j = 0; j < N; j++) {
        C_new[i][j] = 0.;
      }
    }

    // Inicializar uma concentração alta no centro
    C[N/2][N/2] = 1.0;

    // Executar as iterações no tempo para a equação de difusão
    diff_eq(C, C_new);

    // Exibir resultado para verificação
    printf("Concentração final no centro: %f\n", C[N/2][N/2]);
    return 0;
}

Overwriting difusao.c


In [79]:
!rm difusao.x
!gcc difusao.c -o difusao.x
!time ./difusao.x

interacao 0 - diferenca=2.00401e-09
interacao 100 - diferenca=1.23248e-09
interacao 200 - diferenca=7.81794e-10
interacao 300 - diferenca=5.11528e-10
interacao 400 - diferenca=4.21632e-10
Concentração final no centro: 0.216512

real	0m36.649s
user	0m36.272s
sys	0m0.057s


## Implementação com OpenMP

In [82]:
%%writefile difusao_OpenMP.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#define N 2000  // Tamanho da grade
#define T 500 // Número de iterações no tempo
#define D 0.1  // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

void diff_eq(double **C, double **C_new) { //diff_eq(double C[N][N], double C_new[N][N]) {
    int i, j;

    for (int t = 0; t < T; t++) {

        #pragma omp parallel for private(i, j) shared(C_new, C)
        for (i = 1; i < N - 1; i++) {
            for (j = 1; j < N - 1; j++) {
                C_new[i][j] = C[i][j] + D * DELTA_T * (
                    (C[i+1][j] + C[i-1][j] + C[i][j+1] + C[i][j-1] - 4 * C[i][j]) / (DELTA_X * DELTA_X)
                );
            }
        }
        // Atualizar matriz para a próxima iteração
        double difmedio = 0.;

        //Operacao de reducao por soma
        #pragma omp parallel private(i, j) shared(C_new, C) \
          reduction (+:difmedio)
        #pragma omp for
        for ( i = 1; i < N - 1; i++) {
            for ( j = 1; j < N - 1; j++) {
                difmedio = difmedio + fabs(C_new[i][j] - C[i][j]);
                C[i][j] = C_new[i][j];
            }
        }
        if ((t%100) == 0)
          printf("interacao %d - diferenca=%g\n", t, difmedio/((N-2)*(N-2)));
    }
}

int main() {

    int i, j;

    // Concentração inicial
    double **C = (double **)malloc(N * sizeof(double *));
    if (C == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (i = 0; i < N; i++) {
      C[i] = (double *)malloc(N * sizeof(double));
      if (C[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }


    for (i = 0; i < N; i++) {
      for (j = 0; j < N; j++) {
        C[i][j] = 0.;
      }
    }

    // Concentração para a próxima iteração
    double **C_new = (double **)malloc(N * sizeof(double *));
    if (C_new == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (i = 0; i < N; i++) {
      C_new[i] = (double *)malloc(N * sizeof(double));
      if (C_new[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }

    for (i = 0; i < N; i++) {
      for (j = 0; j < N; j++) {
        C_new[i][j] = 0.;
      }
    }

    // Inicializar uma concentração alta no centro
    C[N/2][N/2] = 1.0;

    // Executar as iterações no tempo para a equação de difusão
    diff_eq(C, C_new);

    // Exibir resultado para verificação
    printf("Concentração final no centro: %f\n", C[N/2][N/2]);
    return 0;
}

Overwriting difusao_OpenMP.c


In [83]:
!rm difusao_OpenMP.x
!gcc -fopenmp -o difusao_OpenMP.x difusao_OpenMP.c
!time ./difusao_OpenMP.x

interacao 0 - diferenca=2.00401e-09
interacao 100 - diferenca=1.23248e-09
interacao 200 - diferenca=7.81794e-10
interacao 300 - diferenca=5.11528e-10
interacao 400 - diferenca=4.21632e-10
Concentração final no centro: 0.216512

real	0m34.079s
user	1m0.198s
sys	0m0.106s
